In [58]:
!pip install eli5

    100% |████████████████████████████████| 102kB 3.8MB/s 
    100% |████████████████████████████████| 51kB 21.0MB/s 
  Running setup.py bdist_wheel for tabulate ... - \ done
  Stored in directory: /root/.cache/pip/wheels/2a/85/33/2f6da85d5f10614cbe5a625eab3b3aebfdf43e7b857f25f829
Successfully built tabulate


In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision

# GPUの確認
import torch
use_gpu = torch.cuda.is_available()
print(use_gpu)

In [1]:
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.cross_validation import *
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import io
# import pandas_profiling as pdp
import re

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (1).csv


In [0]:
# Load the data
df_data = pd.read_csv('train.csv', header=0)
df_submission = pd.read_csv('test.csv', header=0)
df_both = df_data.append(df_submission)

In [86]:
print(df_both.columns)
print(df_both.describe())

Index(['Age', 'Cabin', 'Embarked', 'Fare', 'Name', 'Parch', 'PassengerId',
       'Pclass', 'Sex', 'SibSp', 'Survived', 'Ticket'],
      dtype='object')
               Age         Fare        Parch  PassengerId       Pclass  \
count  1046.000000  1308.000000  1309.000000  1309.000000  1309.000000   
mean     29.881138    33.295479     0.385027   655.000000     2.294882   
std      14.413493    51.758668     0.865560   378.020061     0.837836   
min       0.170000     0.000000     0.000000     1.000000     1.000000   
25%      21.000000     7.895800     0.000000   328.000000     2.000000   
50%      28.000000    14.454200     0.000000   655.000000     3.000000   
75%      39.000000    31.275000     0.000000   982.000000     3.000000   
max      80.000000   512.329200     9.000000  1309.000000     3.000000   

             SibSp    Survived  
count  1309.000000  891.000000  
mean      0.498854    0.383838  
std       1.041658    0.486592  
min       0.000000    0.000000  
25%       0.000

In [0]:
# 変数を作成する
# Cabinにいたか
df_both['in_cabin'] = 1
df_both.loc[df_both['Cabin'].isnull(), 'in_cabin'] = 0


# Cabinを複数人でシェアしていたか
df_c = df_both[['Cabin']].copy()
df_c['cabin_shared'] = 1
df_c = df_c.groupby('Cabin').sum().reset_index()

df_both = pd.merge(df_both, df_c, how='left', on=['Cabin'])
df_both['cabin_shared'].fillna(0, inplace=True)
df_both['cabin_shared'] = df_both['cabin_shared'].apply(lambda x : 2 if x > 1 else 0)


# 家族数
df_both['familySize'] = df_both['SibSp'] + df_both['Parch'] + 1
df_both['is_alone'] = df_both['familySize'].apply(lambda x: 1 if x == 1 else 0)

# 敬称を抽出
df_both['Salutation'] = 'others'
df_both.loc[df_both['Name'].apply(lambda x: ', Miss.' in x), 'Salutation'] = 'Miss'
df_both.loc[df_both['Name'].apply(lambda x: ', Mlle.' in x), 'Salutation'] = 'Miss'
df_both.loc[df_both['Name'].apply(lambda x: ', Ms.' in x), 'Salutation'] = 'Miss'
df_both.loc[df_both['Name'].apply(lambda x: ', Mr.' in x), 'Salutation'] = 'Mr' 
df_both.loc[df_both['Name'].apply(lambda x: ', Mrs.' in x), 'Salutation'] = 'Mrs'
df_both.loc[df_both['Name'].apply(lambda x: ', Mme' in x), 'Salutation'] = 'Mrs'
df_both.loc[df_both['Name'].apply(lambda x: ', Sir' in x), 'Salutation'] = 'Sir'
df_both.loc[df_both['Name'].apply(lambda x: ', Master' in x), 'Salutation'] = 'Master'
df_both.loc[df_both['Name'].apply(lambda x: ', Rev.' in x), 'Salutation'] = 'Rev'
df_both.loc[df_both['Name'].apply(lambda x: ', Don.' in x), 'Salutation'] = 'Rev'
# df_both.loc[df_both['Name'].apply(lambda x: ', Dr.' in x), 'Salutation'] = 'Dr'

# Golden Ticket
df_both['golden_ticket'] = df_both['Ticket'].apply(lambda x: 1 
                                                   if (len(x) == 4) 
                                                   or (len(x) == 5 and x[0] in ('1', '2'))
                                                   or (len(x) == 6 and x[0] in ('3'))
                                                   or (len(x) == 7 and x.startswith('PP'))
                                                   or (len(x) == 7 and x.startswith('C '))
                                                   or (len(x) == 9 and x.startswith('C.'))
                                                   or (len(x) == 10 and x.startswith('C.'))
                                                   or x.startswith('PC ')
                                                   or x.startswith('STON/')
                                                   else 0)

# Embarked
df_both['Embarked_n'] = 0
df_both.loc[df_both['Embarked'] == 'Q', 'Embarked_n'] = 1
df_both.loc[df_both['Embarked'] == 'C', 'Embarked_n'] = 2

# 家族属性
df_both['is_family_boy'] = df_both.apply(lambda row: 1 if row['Salutation'] == 'Master' and row['familySize'] in (2, 3, 4) else 0, axis=1)
df_both['is_large_family'] = df_both.apply(lambda row: 1 if row['familySize'] > 4 else 0, axis=1)

# Cabin x Salutation
# df_both['cabin_shared_mr'] = df_both.apply(lambda row: 1 if row['Salutation'] == 'Mr' and row['cabin_shared'] == 2 else 0, axis=1)

In [88]:
# 敬称を平均生存率への影響度に置き換える
salutations = list(set(df_both['Salutation']))
ave_survival_rate = df_both['Survived'].mean()

salutations_impact = {}

for s in salutations:
  impact = df_train[df_both['Salutation'] != s]['Survived'].mean() - ave_survival_rate
  salutations_impact.update({s: impact})

df_both['salutation_impact'] = df_both['Salutation'].apply(lambda x: salutations_impact[x])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [0]:
# チケット価格
df_t = df_both[['Pclass', 'familySize', 'Fare']].groupby(['Pclass', 'familySize'], as_index=False).median().rename(columns={'Fare': 'fare_med'})
df_both = pd.merge(df_both, df_t, how='left', on=['Pclass', 'familySize'])
df_both['high_fare'] = df_both.apply(lambda row: 1 if row['Fare'] > row['fare_med'] else 0, axis=1)

In [90]:
df_both.columns

Index(['Age', 'Cabin', 'Embarked', 'Fare', 'Name', 'Parch', 'PassengerId',
       'Pclass', 'Sex', 'SibSp', 'Survived', 'Ticket', 'in_cabin',
       'cabin_shared', 'familySize', 'is_alone', 'Salutation', 'golden_ticket',
       'Embarked_n', 'is_family_boy', 'is_large_family', 'salutation_impact',
       'fare_med', 'high_fare'],
      dtype='object')

In [0]:
nonnumeric_columns = [
    'Sex'
]
feature_columns_to_use = [
    'Pclass', 
    'golden_ticket',
    'is_family_boy',
    'cabin_shared',
    'high_fare',
    'is_large_family',
    # 'Embarked_n', 
    'salutation_impact'
] + nonnumeric_columns

In [92]:
cut_vals = [[c] for c in feature_columns_to_use]

for c in cut_vals:
  print(df_both[c + ['Survived']].groupby(c).agg({'Survived': ['mean', 'count']}))

        Survived      
            mean count
Pclass                
1       0.629630   216
2       0.472826   184
3       0.242363   491
               Survived      
                   mean count
golden_ticket                
0              0.375385   325
1              0.388693   566
               Survived      
                   mean count
is_family_boy                
0              0.368239   869
1              1.000000    22
              Survived      
                  mean count
cabin_shared                
0             0.321522   762
2             0.751938   129
           Survived      
               mean count
high_fare                
0          0.390438   502
1          0.375321   389
                 Survived      
                     mean count
is_large_family                
0                0.400483   829
1                0.161290    62
                   Survived      
                       mean count
salutation_impact                
-0.083555          0.7027

In [93]:
df_both[feature_columns_to_use].head()

,Pclass,golden_ticket,is_family_boy,cabin_shared,high_fare,is_large_family,salutation_impact,Sex
0,3,0,0,0,0,0,0.314023,male
1,1,1,0,2,0,0,-0.067499,female
2,3,1,0,0,1,0,-0.083555,female
3,1,0,0,2,0,0,-0.067499,female
4,3,1,0,0,1,0,0.314023,male


In [0]:
le = LabelEncoder()
for feature in nonnumeric_columns:
    df_both[feature] = le.fit_transform(df_both[feature])

In [95]:
df_both.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,...,familySize,is_alone,Salutation,golden_ticket,Embarked_n,is_family_boy,is_large_family,salutation_impact,fare_med,high_fare
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,1,...,2,0,Mr,0,0,0,0,0.314023,14.4542,0
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,...,2,0,Mrs,1,2,0,0,-0.067499,71.2833,0
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,0,0,...,1,1,Miss,1,0,0,0,-0.083555,7.8542,1
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,0,1,...,2,0,Mrs,0,0,0,0,-0.067499,71.2833,0
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,1,0,...,1,1,Mr,1,0,0,0,0.314023,7.8542,1


In [0]:
df_data = df_both[df_both['Survived'].notnull()].copy()
df_submission = df_both[df_both['Survived'].isnull()].copy()

X = df_data[feature_columns_to_use]
y = df_data['Survived']

from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.2, random_state=0)

In [98]:
X_train

,Pclass,golden_ticket,is_family_boy,cabin_shared,high_fare,is_large_family,salutation_impact,Sex
140,3,1,0,0,0,0,-0.067499,0
439,2,1,0,0,0,0,0.314023,1
817,2,0,0,0,1,0,0.314023,1
378,3,1,0,0,0,0,0.314023,1
491,3,0,0,0,0,0,0.314023,1
331,1,0,0,2,0,0,0.314023,1
588,3,1,0,0,1,0,0.314023,1
358,3,1,0,0,1,0,-0.083555,0
674,2,0,0,0,0,0,0.314023,1
162,3,1,0,0,0,0,0.314023,1


In [99]:
y_train

140    0.0
439    0.0
817    0.0
378    0.0
491    0.0
331    0.0
588    0.0
358    1.0
674    0.0
162    0.0
580    1.0
402    0.0
671    0.0
875    1.0
727    1.0
428    0.0
54     0.0
45     0.0
172    1.0
447    1.0
597    0.0
204    1.0
240    0.0
672    0.0
39     1.0
283    1.0
308    0.0
18     0.0
769    0.0
156    1.0
      ... 
72     0.0
845    0.0
537    1.0
677    1.0
849    1.0
874    1.0
174    0.0
87     0.0
551    0.0
486    1.0
705    0.0
314    0.0
396    0.0
600    1.0
472    1.0
70     0.0
599    1.0
804    1.0
754    1.0
277    0.0
723    0.0
9      1.0
359    1.0
707    1.0
763    1.0
835    1.0
192    1.0
629    0.0
559    1.0
684    0.0
Name: Survived, Length: 712, dtype: float64

In [100]:
# scikit-learn を用いた交差検証（Cross-validation）とハイパーパラメータのチューニング（grid search）
# https://qiita.com/tomov3/items/039d4271ed30490edf7b

xgb_model = xgb.XGBClassifier()
xgb_params = {
    'objective': ['binary:logistic'],
    'learning_rate': [0.1, 0.2],     # so called `eta` value
    'gamma': [1],
    'max_depth': [4, 5],
    'min_child_weight': [10, 15],
    'silent': [1],
    'subsample': [0.5, 0.6],
    'colsample_bytree': [0.7, 0.8],
    'n_estimators': [500, 700],     # number of trees, change it to 1000 for better results
    'seed': [1337]
}

rf_model = RandomForestClassifier()
rf_params = {
    'n_estimators'      : [700, 800],
    'max_features'      : ['auto'],
    'criterion'         : ['entropy', 'gini'],
    'random_state'      : [0],
    'n_jobs'            : [5],
    'min_samples_split' : [12, 15],
    'max_depth'         : [10, 12, 15],
}

model = rf_model
params = rf_params

clf = GridSearchCV(model, params, n_jobs=5, 
                   cv=StratifiedKFold(y_train, n_folds=5, shuffle=True), 
                   # cv=5,
                   scoring='roc_auc',
                   verbose=2, refit=True)

clf.fit(X_train, y_train)

best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('Raw AUC score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] criterion=entropy, max_depth=10, max_features=auto, min_samples_split=12, n_estimators=700, n_jobs=5, random_state=0 
[CV] criterion=entropy, max_depth=10, max_features=auto, min_samples_split=12, n_estimators=700, n_jobs=5, random_state=0 
[CV] criterion=entropy, max_depth=10, max_features=auto, min_samples_split=12, n_estimators=700, n_jobs=5, random_state=0 
[CV] criterion=entropy, max_depth=10, max_features=auto, min_samples_split=12, n_estimators=700, n_jobs=5, random_state=0 
[CV] criterion=entropy, max_depth=10, max_features=auto, min_samples_split=12, n_estimators=700, n_jobs=5, random_state=0 
[CV]  criterion=entropy, max_depth=10, max_features=auto, min_samples_split=12, n_estimators=700, n_jobs=5, random_state=0 -   3.4s
[CV]  criterion=entropy, max_depth=10, max_features=auto, min_samples_split=12, n_estimators=700, n_jobs=5, random_state=0 -   3.4s
[CV] criterion=entropy, max_depth=10, max_features=auto, mi

[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:   36.5s


[CV]  criterion=entropy, max_depth=12, max_features=auto, min_samples_split=15, n_estimators=700, n_jobs=5, random_state=0 -   5.6s
[CV] criterion=entropy, max_depth=12, max_features=auto, min_samples_split=15, n_estimators=800, n_jobs=5, random_state=0 
[CV]  criterion=entropy, max_depth=12, max_features=auto, min_samples_split=15, n_estimators=700, n_jobs=5, random_state=0 -   5.5s
[CV] criterion=entropy, max_depth=12, max_features=auto, min_samples_split=15, n_estimators=800, n_jobs=5, random_state=0 
[CV]  criterion=entropy, max_depth=12, max_features=auto, min_samples_split=15, n_estimators=700, n_jobs=5, random_state=0 -   5.3s
[CV] criterion=entropy, max_depth=12, max_features=auto, min_samples_split=15, n_estimators=800, n_jobs=5, random_state=0 
[CV]  criterion=entropy, max_depth=12, max_features=auto, min_samples_split=15, n_estimators=800, n_jobs=5, random_state=0 -   6.0s
[CV] criterion=entropy, max_depth=15, max_features=auto, min_samples_split=12, n_estimators=700, n_jobs

[Parallel(n_jobs=5)]: Done 120 out of 120 | elapsed:  2.4min finished


Raw AUC score: 0.8594670471341336
criterion: 'entropy'
max_depth: 12
max_features: 'auto'
min_samples_split: 12
n_estimators: 700
n_jobs: 5
random_state: 0


In [101]:
thres = [0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54]

df_test_result = X_test.copy()
df_test_result['Survived'] = y_test
df_test_result['score'] = [p[1] for p in clf.predict_proba(X_test)]

best_thres = thres[0]
best_score = 0
for t in thres:
  df_test_result['pred'] = df_test_result['score'].apply(lambda x: int(x + t))

  df_test_result['match'] = df_test_result.apply(lambda row: 1 if row['pred'] == row['Survived'] else 0, axis=1)
  score = df_test_result['match'].mean()
  print(t, score)
  
  if score > best_score:
    best_thres = t
    best_score = score
    print('best thres is updated to {}'.format(t))

df_test_result['pred'] = df_test_result['score'].apply(lambda x: int(x + best_thres))

0.46 0.8379888268156425
best thres is updated to 0.46
0.47 0.8603351955307262
best thres is updated to 0.47
0.48 0.8603351955307262
0.49 0.8491620111731844
0.5 0.8491620111731844
0.51 0.8435754189944135
0.52 0.8435754189944135
0.53 0.8435754189944135
0.54 0.8435754189944135


In [79]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(clf, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names=X_test.columns.tolist())

Weight,Feature
0.0928 ± 0.0296,salutation_impact
0.0842 ± 0.0135,Pclass
0.0459 ± 0.0308,Sex
0.0240 ± 0.0179,cabin_shared
0.0180 ± 0.0092,golden_ticket
0.0145 ± 0.0085,is_large_family
0.0087 ± 0.0129,high_fare
0.0083 ± 0.0054,is_family_boy


In [78]:
for c in feature_columns_to_use:
  print(df_test_result[[c, 'Survived', 'match']].groupby([c, 'Survived']).agg({'match': ['mean', 'sum', 'count']}))

                    match          
                     mean sum count
Pclass Survived                    
1      0.0       0.785714  11    14
       1.0       0.812500  26    32
2      0.0       0.900000  18    20
       1.0       0.944444  17    18
3      0.0       0.855263  65    76
       1.0       0.736842  14    19
                           match          
                            mean sum count
golden_ticket Survived                    
0             0.0       0.916667  33    36
              1.0       0.920000  23    25
1             0.0       0.824324  61    74
              1.0       0.772727  34    44
                           match          
                            mean sum count
is_family_boy Survived                    
0             0.0       0.854545  94   110
              1.0       0.818182  54    66
1             1.0       1.000000   3     3
                          match          
                           mean sum count
cabin_shared Survived            

In [0]:
thres = 0.48
# thres = best_thres
df_submission['Survived'] = [int(p[1] + thres) for p in clf.predict_proba(df_submission[feature_columns_to_use])]

In [0]:
df_submission[['PassengerId', 'Survived']].to_csv('submission.csv', index=False)
files.download('submission.csv')

In [0]:
df_test_result.to_csv('detail.csv', index=False)
files.download('detail.csv')